## Find routes that cross multiple districts

In [26]:
import A1_provider_prep
import A2_other
import A3_analysis
import dask.dataframe as dd
import dask_geopandas as dg
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.wkt
import shared_utils
from calitp.sql import to_snakecase
from shared_utils import geography_utils, utils

In [27]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [28]:
one_dist_routes, multi_dist_routes, all_routes = A2_other.find_multi_district_routes()

In [29]:
districts = A1_provider_prep.get_districts()

In [30]:
# districts.crs == unique_routes.crs

In [31]:
# type(districts), type(unique_routes)

### Sjoin - Test
* Ended up using clip instead.
* Place unique routes on the L because I want the routes that fall within each district.
* Intersects only returns routes that are 100% in a district. Should I used something else (within,contains,etc)?
* Sjoin does *not* change geometry.

In [32]:
def sjoin_route_district(district_df: gpd.GeoDataFrame):
    """
    Find which routes fall 100% neatly in an district and
    which don't.
    """
    unique_routes = A3_other.load_unique_routes_df()
    sjoin = gpd.sjoin(
        unique_routes, district_df, how="inner", predicate="intersects"
    ).drop(columns="index_right")

    # Get route length after doing sjoin
    sjoin = sjoin.assign(
        sjoin_route_length=sjoin.geometry.to_crs(geography_utils.CA_StatePlane).length
    )

    # Get %
    sjoin["route_percentage"] = (
        (sjoin["sjoin_route_length"] / sjoin["original_route_length"]) * 100
    ).astype("int64")

    return sjoin

In [33]:
def complete_sjoin_route_district() -> dg.GeoDataFrame:
    """
    Find which routes fall 100% neatly in an district and
    which cross district boundaries. Stack results altogether.
    Filter out for routes that fall in mulitple districts.
    """
    district_df = A1_provider_prep.get_districts()

    full_gdf = pd.DataFrame()

    for i in [*range(1, 13, 1)]:
        result = sjoin_route_district(district_df[district_df.district == i])

        full_gdf = dd.multi.concat([full_gdf, result], axis=0)

    full_gdf = full_gdf.compute()

    return full_gdf

In [34]:
# sjoin_test =

#### Sjoin Testing w/ D4

In [35]:
# Subset to D4
d4 = districts[districts.district == 4]

In [36]:
# Sjoin
# Should it be within? Gives me only routes that fall 100% in d4
# d4_sjoin = gpd.sjoin(unique_routes, d4, how="inner", predicate="intersects").drop(
#     columns="index_right"
# )

In [37]:
# d4_sjoin2.drop(columns = ['geometry']).sort_values("percentage_route_covered")

In [38]:
# d4_sjoin.shape

In [39]:
# d4_sjoin.agency.value_counts()

### Clip
* How can I keep district info? Or incorporate which district the route falls in?
* Is unique routes and district in the right order, L and R?
* How can there be routes that don't add up to even 90% after grouping by long route name??
* A route must fall under ONE district completely?

In [40]:
# 9:50 am -  9:52 took less than a minute
# clip_test = A2_other.complete_clip_route_district()

In [41]:
# 5:16
routes_in_1d, routes_in_2plus_d, all_routes =  A2_other.find_multi_district_routes()

In [42]:
routes_in_1d.drop(columns = ["geometry"]).sample()


,itp_id,route_id,route_name,agency,original_route_length,long_route_name,clipped_route_length,route_percentage,District
2399,284,580,DTC - SECTION/ORO,San Joaquin Regional Transit District,30489.83,DTC - SECTION/ORO 580 San Joaquin Regional Transit District,30489.83,100,D-10


In [43]:
# routes_in_2plus_d.drop(columns = ["geometry"]).sort_values("long_route_name")

In [44]:
# How are there more routes now?
# Original unique routes only has 2829 rows, now there are  2914 rows
# Only 2821 unique long route names here but original unique routes has 2829 unique routes
# clip_test.shape, clip_test.long_route_name.nunique()

In [45]:
# 85 more rows
# len(clip_test) - len(unique_routes)

In [46]:
# 8 routes different, unique_routes have some routes that go to Arizona
# clip_routes_list = set(clip_test.long_route_name.unique().tolist())
# unique_routes_list = set(unique_routes.long_route_name.unique().tolist())

In [47]:
# unique_routes_list - clip_routes_list

In [48]:
# Find routes that have 1+ rows
# Having 1+ row would mean this route goes across 1+ district

In [49]:
# 10:21am, 10:23
# one_district_routes, multi_district_routes, all_routes = A3_other.find_multi_district_routes()

In [50]:
len(one_district_routes) + len(multi_district_routes) == len(all_routes)

NameError: name 'one_district_routes' is not defined

In [ ]:
multi_district_routes.long_route_name.nunique()

In [ ]:
# multi_district_routes.drop(columns = ["geometry"]).sort_values("long_route_name")

In [ ]:
# df2.groupby(["long_route_name"]).agg({'route_percentage':'sum'})

In [ ]:
# value_counts_d4 = value_counts_df(clip_test, 'long_route_name')

In [ ]:
# value_counts_d4.sort_values('total', ascending = False)

In [ ]:
# Find routes w/ 1+ row
# routes_with_2_plus = (value_counts_d4[value_counts_d4.total > 1])[['long_route_name']]

# Place routes into a list
# routes_with_2_plus_list = routes_with_2_plus.long_route_name.tolist()

In [ ]:
# len(routes_with_2_plus_list)

In [ ]:
# Filter out for these routes 
# routes_in_2_districts_clipped = clip_test[clip_test.long_route_name.isin(routes_with_2_plus_list)]

In [ ]:
# routes_in_1_district_clipped = clip_test[~clip_test.long_route_name.isin(routes_with_2_plus_list)]

In [ ]:
# len(routes_in_1_district_clipped) + len(routes_in_2_districts_clipped) == len(clip_test)

In [ ]:
# Filter out for these routes 
# routes_in_2_districts_original = unique_routes[unique_routes.long_route_name.isin(routes_with_2_plus_list)]

In [ ]:
n = districts.explore(
    "district",
    tiles = "CartoDB positron",
    cmap = "Blues",
    name="districts",
    width=800,
    height=400,
)

# Clipped geometry
n = multi_district_routes.explore("long_route_name",
    m=n, legend = False
)

# Clipped, altered geometry
#n = routes_in_2_districts_clipped.explore(
   # m=n, color = "red", style_kwds = {'opacity': 0.5, 'weight':3})

In [ ]:
# n

#### Clip Testing w/ D4

In [ ]:
# Try with clip instead
d4_clip = gpd.clip(unique_routes, d4)

In [ ]:
d4_clip2 = d4_clip.assign(
    clip_route_length=d4_clip.geometry.to_crs(geography_utils.CA_StatePlane).length
)

In [ ]:
# Get %
d4_clip2["route_percentage"] = (
    (d4_clip2["clip_route_length"] / d4_clip2["original_route_length"]) * 100
).astype("int64")

In [ ]:
d4_clip2.shape

In [ ]:
# d4_clip2.route_percentage.value_counts()

In [ ]:
# Filter for only routes that have whatever percent in D4
low_coverage_d4 = d4_clip2.loc[d4_clip2.route_percentage < 85].sort_values(
    "route_percentage"
)

In [ ]:
# Grab routes that is 85% or less of original route length into a list
low_coverage_d4_list = low_coverage_d4.long_route_name.unique().tolist()

In [ ]:
# Filter out for the routes above in original unique_routes
# So I can map original route geometries
low_coverage_d4_og_geometry = (
    unique_routes[unique_routes["long_route_name"].isin(low_coverage_d4_list)]
).reset_index(drop=True)

In [ ]:
low_coverage_d4_og_geometry.shape

In [ ]:
# https://stackoverflow.com/questions/73767559/geopandas-explore-how-to-plot-more-than-one-layer
# Routes plotted with original geometry
# Certain routes in here are clearly only in D4 yet their percentage is low...
m = d4.explore(
    color="#F4D837",
    tiles="CartoDB positron",
    name="D4",
    width=800,
    height=500,
)

# Route: original length
m = low_coverage_d4_og_geometry.explore(
    m=m,
)

# Route: clipped length
m = low_coverage_d4.explore(
    m=m, color = "red", style_kwds = {'opacity': 0.5, 'weight':3})

In [ ]:
m

In [ ]:
# check to see if 2 columns are the same
# d4_clip_routes_set = set(low_coverage_d4_list)
# low_percent_routes_set = set(low_percent_routes_list)

In [ ]:
# d4_clip_routes_set - low_percent_routes_set

In [ ]:
# Check some of the routes that are found in d4 test versus clip_test
# How is CC RIder 231 now 95% in a certain district?
# clip_test.drop(columns = ["geometry"])[clip_test.long_route_name == 'CC Rider 231  Mendocino Transit Authority']

In [ ]:
# d4_clip2.drop(columns = ["geometry"])[d4_clip2.long_route_name == 'CC Rider 231  Mendocino Transit Authority']

In [ ]:
# clip_test.drop(columns = ["geometry"])[clip_test.long_route_name == 'South Coast / Santa Rosa 224  Mendocino Transit Authority']

In [ ]:
# Check routes with test d4
# d4_clip2.drop(columns = ["geometry"])[d4_clip2.long_route_name == 'South Coast / Santa Rosa 224  Mendocino Transit Authority']

### Scratch Area

In [ ]:
def clip_route_district():
    """
    Find which routes fall 100% neatly in an district and
    which cross district boundaries by each district.
    Stack the seperated district results back together.
    """
    # Load unique routes & districts
    unique_routes = A3_other.load_unique_routes_df()
    district_df = A1_provider_prep.get_districts()
    
    # Clip routes against a district
    clipped = gpd.clip(unique_routes, district_df)

    # Get route length after doing clip
    clipped = clipped.assign(
        clipped_route_length=clipped.geometry.to_crs(
            geography_utils.CA_StatePlane
        ).length
    )

    # Get %
    clipped["route_percentage"] = (
        (clipped["clipped_route_length"] / clipped["original_route_length"]) * 100
    ).astype("int64")

    return clipped

In [ ]:
# clip_d1_d4 = complete_clip_route_district()